In [7]:
import os
import json
from PIL import Image
from rfdetr import RFDETRBase # Or other specific RF-DETR model class
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

# --- Configuration ---
IMAGE_FOLDER = "1-class-weapon-dataset-1/test" # !!! CHANGE THIS !!!
GROUND_TRUTH_ANNOTATION_FILE = "1-class-weapon-dataset-1/test/_annotations.coco.json" # !!! CHANGE THIS !!! Needed for evaluation
OUTPUT_PREDICTIONS_FILE = "rf_detr_predictions_1class.json" # Optional: Save predictions

# --- Load RF-DETR Model ---
# Load the base model (or specify weights if you have a fine-tuned model)
print("Loading RF-DETR model...")
model = RFDETRBase(pretrain_weights='output3/checkpoint_best_ema.pth')

print("Model loaded.")

# --- Prepare for Predictions & Evaluation ---
all_predictions_coco_format = []
image_id_map = {} # Needed if your annotations use COCO image IDs

# --- Load Ground Truth (Needed for Evaluation) ---
# This assumes your annotations are in COCO format. Adjust if necessary.
try:
    print(f"Loading ground truth annotations from: {GROUND_TRUTH_ANNOTATION_FILE}")
    cocoGt = COCO(GROUND_TRUTH_ANNOTATION_FILE)

    # Create a mapping from filename to COCO image ID (if needed)
    for img_info in cocoGt.dataset['images']:
        image_id_map[img_info['file_name']] = img_info['id']
    print("Ground truth loaded.")

except FileNotFoundError:
    print(f"ERROR: Ground truth annotation file not found at {GROUND_TRUTH_ANNOTATION_FILE}")
    print("Evaluation cannot be performed without ground truth.")
    cocoGt = None
except Exception as e:
    print(f"ERROR loading ground truth annotations: {e}")
    cocoGt = None


# --- Iterate Through Images and Predict ---
print(f"Processing images from folder: {IMAGE_FOLDER}")
for filename in os.listdir(IMAGE_FOLDER):
    # Check if it's an image file (add more extensions if needed)
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
        image_path = os.path.join(IMAGE_FOLDER, filename)
        # print(f"  Predicting on: {filename}")

        try:
            # Load Image
            image = Image.open(image_path).convert("RGB") # Ensure image is RGB
            image_width, image_height = image.size

            # Perform Inference
            # Note: The 'predict' method might return detections in a specific format.
            # You'll need to inspect the 'detections' object structure.
            # Common attributes might be .xyxy, .confidence, .class_id
            detections = model.predict(image, threshold=0.5) # Adjust threshold as needed

            # --- Format Predictions for COCOeval ---
            # This part requires knowing the exact structure of 'detections'
            # and having the 'image_id' from your ground truth annotations.
            if cocoGt and filename in image_id_map:
                current_image_id = image_id_map[filename]

                # Example loop - **ADAPT BASED ON `detections` STRUCTURE**
                for i in range(len(detections)): # Assuming detections is indexable
                   # Example accessors - REPLACE with actual attributes/methods
                   # Make sure bbox format is [x, y, width, height]
                   box = detections.xyxy[i] # Assuming xyxy format [xmin, ymin, xmax, ymax]
                   score = detections.confidence[i] # Assuming confidence attribute
                   class_id = detections.class_id[i] # Assuming class_id attribute

                   # Convert bbox from [xmin, ymin, xmax, ymax] to [x, y, width, height]
                   x = float(box[0])
                   y = float(box[1])
                   width = float(box[2] - box[0])
                   height = float(box[3] - box[1])
                   coco_bbox = [x, y, width, height]

                   # Get the category ID that matches your ground truth file's categories
                   # This might require a mapping if your model outputs names or different IDs
                   # For simplicity, assuming model class_id directly maps to COCO category_id
                   category_id = int(class_id) # Adapt if needed

                   prediction_entry = {
                       "image_id": current_image_id,
                       "category_id": category_id, # Ensure this matches GT category IDs
                       "bbox": coco_bbox,
                       "score": float(score)
                   }
                   all_predictions_coco_format.append(prediction_entry)
            else:
                 if not cocoGt:
                     print(f"  Skipping prediction formatting for {filename} (Ground truth not loaded)")
                 else:
                     print(f"  Skipping prediction formatting for {filename} (Filename not found in ground truth image map)")


        except Exception as e:
            print(f"  ERROR processing {filename}: {e}")

print("Finished processing images.")

# --- Save Predictions (Optional) ---
if all_predictions_coco_format:
    print(f"Saving {len(all_predictions_coco_format)} predictions to {OUTPUT_PREDICTIONS_FILE}...")
    with open(OUTPUT_PREDICTIONS_FILE, 'w') as f:
        json.dump(all_predictions_coco_format, f, indent=4)
    print("Predictions saved.")

# --- Calculate Evaluation Metrics (Requires Ground Truth and Predictions) ---
if cocoGt and all_predictions_coco_format:
    print("\n--- Running COCO Evaluation ---")
    # Load predictions into COCO format
    cocoDt = cocoGt.loadRes(all_predictions_coco_format)

    # Initialize evaluator
    cocoEval = COCOeval(cocoGt, cocoDt, iouType='bbox') # Use 'bbox' for detection

    # Run evaluation
    cocoEval.evaluate()
    cocoEval.accumulate()

    # Print summary
    print("\nCOCO Evaluation Summary:")
    cocoEval.summarize()
    print("--- Evaluation Complete ---")

elif not cocoGt:
    print("\nEvaluation skipped because ground truth annotations could not be loaded.")
else:
    print("\nEvaluation skipped because no predictions were successfully formatted.")

Loading RF-DETR model...


num_classes mismatch: pretrain weights has 1 classes, but your model has 90 classes
reinitializing detection head with 1 classes


Loading pretrain weights
Model loaded.
Loading ground truth annotations from: 1-class-weapon-dataset-1/test/_annotations.coco.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Ground truth loaded.
Processing images from folder: 1-class-weapon-dataset-1/test
Finished processing images.
Saving 533 predictions to rf_detr_predictions_1class.json...
Predictions saved.

--- Running COCO Evaluation ---
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.07s).
Accumulating evaluation results...
DONE (t=0.02s).

COCO Evaluation Summary:
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.090
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.233
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.051
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.076
 Aver